In [1]:
!nvidia-smi

Mon Jun 19 20:18:58 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.116.04   Driver Version: 525.116.04   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA RTX A6000    Off  | 00000000:41:00.0 Off |                  Off |
| 30%   31C    P8    19W / 300W |      0MiB / 49140MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
import os 
os.environ["HF_HOME"] ="/fs/nexus-scratch/xhnotic/.cache/huggingface"

In [ ]:
from nltk.tokenize import sent_tokenize
import re

In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM

/nfshomes/xhnotic/miniconda3/envs/llm/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
tokenizer = AutoTokenizer.from_pretrained("eachadea/vicuna-13b-1.1")
model = AutoModelForCausalLM.from_pretrained("eachadea/vicuna-13b-1.1", device_map="auto")

The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.
Loading checkpoint shards: 100%|██████████| 3/3 [01:17<00:00, 25.67s/it]


In [21]:
#input content, output paraphrased content
def paraphrase(content):

    instruction ="USER:\n" + "Paraphrase the following sentences:"
    input_text =  instruction + content
    input_ids = tokenizer.encode(input_text, return_tensors='pt', max_length=1024, truncation=True)
    output = model.generate(input_ids, max_length=1024, do_sample=True, temperature=1, output_scores=True, return_dict_in_generate=True)
    response = tokenizer.decode(output["sequences"][0], skip_special_tokens=True)
    
    # Extract the substring after "ASSISTANT:"
    index = response.find("ASSISTANT:")
    if index != -1:
        result = response[index + len("ASSISTANT:"):].strip()
     
    return result

In [64]:
#input a long string, returns grouped paragraph
def group_string(strings, group_size=10):
    
    grouped_strings = []
    tokenized_strings = sent_tokenize(strings)
    for i in range(0, len(tokenized_strings), group_size):
        group = ''.join(tokenized_strings[i:i+group_size])
        grouped_strings.append(group)
        
    return grouped_strings 

In [ ]:
def article_2_paraphrased(article, group_size=10):
    string_list = group_string(article, group_size=10)
    paraphrased_list = []
    for i in range(len(string_list)):
        paraphrased_string = paraphrase(string_list[i])
        paraphrased_list.append(paraphrased_string)
        
    returnparaphrased_list